In [ ]:
# ---
# jupyter:
#   accelerator: GPU
#   jupytext:
#     cell_metadata_filter: -all
#     formats: nb_python//py:percent,notebooks//ipynb
#     notebook_metadata_filter: allba
#       extension: .p
#       format_name: percent
#       format_version: '1.3'
#       jupytext_version: 1.13.7
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
#   language_info:
#     codemirror_mode:
#       name: ipython
#       version: 3
#     file_extension: .py
#     mimetype: text/x-python
#     name: python
#     nbconvert_exporter: python
#     pygments_lexer: ipython3
#     version: 3.9.17
# ---

# %% [markdown]
# #Habitat-sim ReplicaCAD Quickstart
#
# This brief tutorial demonstrates loading the ReplicaCAD dataset in Habitat-sim from a SceneDataset and rendering a short video of agent navigation with physics simulation.
#

# %%
# @title Path Setup and Imports { display-mode: "form" }
# @markdown (double click to show code).

## [setup]
import os

import git
import magnum as mn

import habitat_sim
from habitat_sim.utils import viz_utils as vut

try:
    import ipywidgets as widgets
    from IPython.display import display as ipydisplay

    # For using jupyter/ipywidget IO components

    HAS_WIDGETS = True
except ImportError:
    HAS_WIDGETS = False

repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
data_path = os.path.join(dir_path, "data")
output_path = os.path.join(
    dir_path, "examples/tutorials/replica_cad_output/"
)  # @param {type:"string"}
os.makedirs(output_path, exist_ok=True)

# define some globals the first time we run.
if "sim" not in globals():
    global sim
    sim = None
    global obj_attr_mgr
    obj_attr_mgr = None
    global stage_attr_mgr
    stage_attr_mgr = None
    global rigid_obj_mgr
    rigid_obj_mgr = None


# %%
# @title Define Configuration Utility Functions { display-mode: "form" }
# @markdown (double click to show code)

# @markdown This cell defines a number of utility functions used throughout the tutorial to make simulator reconstruction easy:
# @markdown - make_cfg
# @markdown - make_default_settings
# @markdown - make_simulator_from_settings


def make_cfg(settings):
    sim_cfg = habitat_sim.SimulatorConfiguration()
    sim_cfg.gpu_device_id = 0
    sim_cfg.scene_dataset_config_file = settings["scene_dataset"]
    sim_cfg.scene_id = settings["scene"]
    sim_cfg.enable_physics = settings["enable_physics"]
    # Specify the location of the scene dataset
    if "scene_dataset_config" in settings:
        sim_cfg.scene_dataset_config_file = settings["scene_dataset_config"]
    if "override_scene_light_defaults" in settings:
        sim_cfg.override_scene_light_defaults = settings[
            "override_scene_light_defaults"
        ]
    if "scene_light_setup" in settings:
        sim_cfg.scene_light_setup = settings["scene_light_setup"]

    # Note: all sensors must have the same resolution
    sensor_specs = []
    color_sensor_1st_person_spec = habitat_sim.CameraSensorSpec()
    color_sensor_1st_person_spec.uuid = "color_sensor_1st_person"
    color_sensor_1st_person_spec.sensor_type = habitat_sim.SensorType.COLOR
    color_sensor_1st_person_spec.resolution = [
        settings["height"],
        settings["width"],
    ]
    color_sensor_1st_person_spec.position = [0.0, settings["sensor_height"], 0.0]
    color_sensor_1st_person_spec.orientation = [
        settings["sensor_pitch"],
        0.0,
        0.0,
    ]
    color_sensor_1st_person_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
    sensor_specs.append(color_sensor_1st_person_spec)

    depth_sensor_spec = habitat_sim.CameraSensorSpec()
    depth_sensor_spec.uuid = "depth_sensor"
    depth_sensor_spec.sensor_type = habitat_sim.SensorType.DEPTH
    depth_sensor_spec.resolution = [settings["height"], settings["width"]]
    depth_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    depth_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
    sensor_specs.append(depth_sensor_spec)

    semantic_sensor_spec = habitat_sim.CameraSensorSpec()
    semantic_sensor_spec.uuid = "semantic_sensor"
    semantic_sensor_spec.sensor_type = habitat_sim.SensorType.SEMANTIC
    semantic_sensor_spec.resolution = [settings["height"], settings["width"]]
    semantic_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    semantic_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE
    sensor_specs.append(semantic_sensor_spec)

    # Here you can specify the amount of displacement in a forward action and the turn angle
    agent_cfg = habitat_sim.agent.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs

    return habitat_sim.Configuration(sim_cfg, [agent_cfg])


def make_default_settings():
    rgb_sensor = True  # @param {type:"boolean"}
    depth_sensor = True  # @param {type:"boolean"}
    semantic_sensor = True  # @param {type:"boolean"}
    settings = {
        "width": 1280,  # Spatial resolution of the observations
        "height": 720,
        "scene_dataset": os.path.join(
            data_path, "replica_cad/replicaCAD.scene_dataset_config.json"
        ),  # dataset path
        "scene": "NONE",  # Scene path
        "default_agent": 0,
        "sensor_height": 1.5,  # Height of sensors in meters
        "sensor_pitch": 0.0,  # sensor pitch (x rotation in rads)
        "color_sensor": rgb_sensor,  # RGB sensor
        "depth_sensor": depth_sensor,  # Depth sensor
        "semantic_sensor": semantic_sensor,  # Semantic sensor
        "seed": 1,
        "enable_physics": True,  # enable dynamics simulation
    }
    return settings


def make_simulator_from_settings(sim_settings):
    cfg = make_cfg(sim_settings)
    # clean-up the current simulator instance if it exists
    global sim
    global obj_attr_mgr
    global prim_attr_mgr
    global stage_attr_mgr
    global rigid_obj_mgr
    global metadata_mediator

    if sim != None:
        sim.close()
    # initialize the simulator
    sim = habitat_sim.Simulator(cfg)
    # Managers of various Attributes templates
    obj_attr_mgr = sim.get_object_template_manager()
    obj_attr_mgr.load_configs(str(os.path.join(data_path, "objects/example_objects")))
    prim_attr_mgr = sim.get_asset_template_manager()
    stage_attr_mgr = sim.get_stage_template_manager()
    # Manager providing access to rigid objects
    rigid_obj_mgr = sim.get_rigid_object_manager()
    # get metadata_mediator
    metadata_mediator = sim.metadata_mediator

    # UI-populated handles used in various cells.  Need to initialize to valid
    # value in case IPyWidgets are not available.
    # Holds the user's desired scene handle
    global selected_scene
    selected_scene = "NONE"


# [/setup]


# %%
# @title Define Simulation Utility Function { display-mode: "form" }
# @markdown (double click to show code)
def simulate(sim, dt=1.0, get_frames=True):
    # simulate dt seconds at 60Hz to the nearest fixed timestep
    print("Simulating {:.3f} world seconds.".format(dt))
    observations = []
    start_time = sim.get_world_time()
    while sim.get_world_time() < start_time + dt:
        sim.step_physics(1.0 / 60.0)
        if get_frames:
            observations.append(sim.get_sensor_observations())
    return observations


# %%
# @title Define GUI Utility Functions { display-mode: "form" }
# @markdown (double click to show code)

# @markdown This cell provides utility functions to build and manage IPyWidget interactive components.


# Event handler for dropdowns displaying file-based object handles
def on_scene_ddl_change(ddl_values):
    global selected_scene
    selected_scene = ddl_values["new"]
    return selected_scene


# Build a dropdown list holding obj_handles and set its event handler
def set_handle_ddl_widget(scene_handles, sel_handle, on_change):
    descStr = "Available Scenes:"
    style = {"description_width": "300px"}
    obj_ddl = widgets.Dropdown(
        options=scene_handles,
        value=sel_handle,
        description=descStr,
        style=style,
        disabled=False,
        layout={"width": "max-content"},
    )

    obj_ddl.observe(on_change, names="value")
    return obj_ddl, sel_handle


def set_button_launcher(desc):
    button = widgets.Button(
        description=desc,
        layout={"width": "max-content"},
    )
    return button


# Builds widget-based UI components
def build_widget_ui(metadata_mediator):
    # Holds the user's desired scene
    global selected_scene

    # All file-based object template handles
    scene_handles = metadata_mediator.get_scene_handles()
    # Set default as first available valid handle, or NONE scene if none are available
    if len(scene_handles) == 0:
        selected_scene = "NONE"
    else:
        # Set default selection to be first valid non-NONE scene (for python consumers)
        for scene_handle in scene_handles:
            if "NONE" not in scene_handle:
                selected_scene = scene_handle
                break

    if not HAS_WIDGETS:
        # If no widgets present, return, using default
        return

    # Construct DDLs and assign event handlers
    # Build widgets
    scene_obj_ddl, selected_scene = set_handle_ddl_widget(
        scene_handles,
        selected_scene,
        on_scene_ddl_change,
    )

    # Display DDLs
    ipydisplay(scene_obj_ddl)


if __name__ == "__main__":
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--no-display", dest="display", action="store_false")
    parser.add_argument("--no-make-video", dest="make_video", action="store_false")
    parser.set_defaults(show_video=True, make_video=True)
    args, _ = parser.parse_known_args()
    show_video = args.display
    display = args.display
    make_video = args.make_video
else:
    show_video = False
    make_video = False
    display = False

from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
# Change to do something like this maybe: https://stackoverflow.com/a/41432704
def save_sample(rgb_obs, semantic_obs, depth_obs, idx, save_path=dir_path + '/output'):
    from habitat_sim.utils.common import d3_40_colors_rgb

    # Create and save the RGB image
    rgb_img = Image.fromarray(rgb_obs, mode="RGBA")
    rgb_img.save(f"{save_path}/rgb_image_{idx}.png")

    # Create and save the semantic image
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    semantic_img.save(f"{save_path}/semantic_image_{idx}.png")

    # Create and save the depth image
    depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
    depth_img.save(f"{save_path}/depth_image_{idx}.png")

# %% [markdown]
# # View ReplicaCAD in Habitat-sim
# Use the code in this section to view assets in the Habitat-sim engine.

# %%
# [initialize]
# @title Initialize Simulator{ display-mode: "form" }

sim_settings = make_default_settings()
make_simulator_from_settings(sim_settings)
# [/initialize]

# %%
# @title Select a SceneInstance: { display-mode: "form" }
# @markdown Select a scene from the dropdown and then run the next cell to load and simulate that scene and produce a visualization of the result.

build_widget_ui(sim.metadata_mediator)

# %% [markdown]
# ## Load the Select Scene and Simulate!
# This cell will load the scene selected above, simulate, and produce a visualization.

# %%
global selected_scene
if sim_settings["scene"] != selected_scene:
    sim_settings["scene"] = selected_scene
    make_simulator_from_settings(sim_settings)

observations = []
translations = []
rotations = []
start_time = sim.get_world_time()
count = 1
sim.agents[0].scene_node.translation = mn.Vector3([-2, 0, 0])
while sim.get_world_time() < start_time + 4.0:
    if count < 40:
        sim.agents[0].scene_node.rotate(mn.Rad(- mn.math.pi_half / 20.0), mn.Vector3(0, 1, 0))
    elif count < 60:
        sim.agents[0].scene_node.translation += np.array([0.3, 0, 0])
        sim.agents[0].scene_node.rotate(mn.Rad(- mn.math.pi_half / 20.0), mn.Vector3(0, 1, 0))
    elif count < 80:
        sim.agents[0].scene_node.translation += np.array([0.0, 0, 0.3])
        sim.agents[0].scene_node.rotate(mn.Rad(- mn.math.pi_half / 20.0), mn.Vector3(0, 1, 0))
    else:
        sim.agents[0].scene_node.translation += np.array([-0.1, 0, -0.1])
    sim.step_physics(1.0 / 30.0)
    if make_video:
        observation = sim.get_sensor_observations()
        observations.append(observation)
        rgb = observation["color_sensor_1st_person"]
        semantic = observation["semantic_sensor"]
        depth = observation["depth_sensor"]
        translation = sim.agents[0].scene_node.translation
        rotation = sim.agents[0].scene_node.rotation
        translations.append(translation)
        rotations.append(rotation)
        if display:
            save_sample(rgb, semantic, depth, count)
            count +=1 

# video rendering of carousel view
# video_prefix = "ReplicaCAD_scene_view"
# if make_video:
#     vut.make_video(
#         observations,
#         "color_sensor_1st_person",
#         "color",
#         output_path + video_prefix,
#         open_vid=show_video,
#         video_dims=[1280, 720],
#     )

In [ ]:
# Example inputs (replace with your data)
width, height = 1280, 720
fov = 90
fx = fy = 0.5 * width / np.tan(np.radians(fov) / 2)
cx, cy = width / 2, height / 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Camera intrinsics and depth-to-3D conversion function
def depth_to_point_cloud(depth, color, fx, fy, cx, cy):
    """
    Convert depth image to point cloud.

    Parameters:
        depth (ndarray): Depth image (H, W) in meters.
        color (ndarray): Color image (H, W, 3).
        fx, fy, cx, cy: Intrinsic parameters.
        
    Returns:
        points (ndarray): Point cloud positions (N, 3).
        colors (ndarray): Point cloud colors (N, 3).
    """
    height, width = depth.shape
    # Create a grid of coordinates corresponding to each pixel
    i, j = np.meshgrid(np.arange(width), np.arange(height), indexing='xy')
    
    # Calculate 3D coordinates in camera space
    x = (i - cx) * depth / fx
    y = (j - cy) * depth / fy
    z = depth

    # Mask valid depth values (> 0)
    valid = depth > 0
    points = np.stack((x[valid], y[valid], z[valid]), axis=-1)
    colors = color[valid]
    return points, colors

# Example inputs (replace with your data)
width, height = sim_settings['width'], sim_settings['height']
fov = float(sim.agents[0]._sensors['color_sensor_1st_person'].hfov)  # degrees
fx = fy = 0.5 * width / np.tan(np.radians(fov) / 2)
cx, cy = width / 2, height / 2

# Convert to point cloud
points, colors = depth_to_point_cloud(depth, rgb[:,:,:3], fx, fy, cx, cy)

# Plot point cloud
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=colors / 255.0, s=0.5)
ax.set_xlabel("X (m)")
ax.set_ylabel("Y (m)")
ax.set_zlabel("Z (m)")
plt.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import cv2

# Camera intrinsics and optimized depth-to-3D conversion
def depth_to_point_cloud_fast(depth, color, fx, fy, cx, cy):
    height, width = depth.shape
    i, j = np.meshgrid(np.arange(width), np.arange(height), indexing='xy')
    x = (i - cx) * depth / fx
    y = (j - cy) * depth / fy
    z = depth
    valid = depth > 0
    points = np.stack((x[valid], y[valid], z[valid]), axis=-1)
    colors = color[valid]
    return points, colors

# Example inputs (replace with your data)
width, height = 1280, 720
fov = 90
fx = fy = 0.5 * width / np.tan(np.radians(fov) / 2)
cx, cy = width / 2, height / 2


# Downsample the depth and color images for performance
depth_resized = cv2.resize(depth, (depth.shape[1] // 2, depth.shape[0] // 2))
rgb_resized = cv2.resize(rgb, (rgb.shape[1] // 2, rgb.shape[0] // 2))

# Convert to point cloud
points, colors = depth_to_point_cloud_with_pose(depth, rgb[:,:,:3], fx, fy, cx, cy, translation, rotation)  # depth_to_point_cloud_fast(depth_resized, rgb_resized, fx, fy, cx, cy)


# Downsample the point cloud
sample_size = int(len(points) * 0.1)  # Keep 10% of the points
indices = np.random.choice(len(points), sample_size, replace=False)
points_sampled = points[indices]
colors_sampled = colors[indices]

# Update layout for better appearance and interaction
fig.update_layout(
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)",
    ),
    title="Interactive 3D Point Cloud",
    margin=dict(l=0, r=0, b=0, t=40)
)
fig = go.Figure(data=[go.Scatter3d(
    x=points_sampled[:, 0],
    y=points_sampled[:, 1],
    z=points_sampled[:, 2],
    mode='markers',
    marker=dict(
        size=1,
        color=colors_sampled,  # Use the sampled colors directly from the color image
        opacity=0.6
    )
)])


# Show the interactive plot
fig.show()


In [ ]:
fig.update_layout(
    scene=dict(
        xaxis_title="X (m)",
        yaxis_title="Y (m)",
        zaxis_title="Z (m)",
    ),
    title="Interactive 3D Point Cloud",
    margin=dict(l=0, r=0, b=0, t=40)
)
fig = go.Figure(data=[go.Scatter3d(
    x=points_sampled[:, 0],
    y=points_sampled[:, 2],
    z=- points_sampled[:, 1],
    mode='markers',
    marker=dict(
        size=1,
        color=colors_sampled,  # Use the sampled colors directly from the color image
        opacity=0.6
    )
)])


# Show the interactive plot
fig.show()

In [ ]:
# After the loop, merged_points and merged_colors hold the combined point cloud
if merged_points:  # Only attempt to merge if there are points
    merged_points = np.vstack(merged_points)
    merged_colors = np.vstack(merged_colors)
    
    # Check if merged_points and merged_colors have the correct shapes
    if merged_points.shape[0] > 0 and merged_colors.shape[0] > 0:
        # Create 3D scatter plot using Plotly's Scatter3d (correct for 3D data)
        fig.update_layout(
            scene=dict(
                xaxis_title="X (m)",
                yaxis_title="Y (m)",
                zaxis_title="Z (m)",
            ),
            title="Interactive 3D Point Cloud",
            margin=dict(l=0, r=0, b=0, t=40)
        )
        fig = go.Figure(data=[go.Scatter3d(
            x=merged_points[:, 0],
            y=merged_points[:, 2],
            z=- merged_points[:, 1],
            mode='markers',
            marker=dict(
                size=1,
                color=merged_colors,  # Use the sampled colors directly from the color image
                opacity=0.6
            )
        )])

        # Show the interactive plot
        fig.show()
    else:
        print("Merged point cloud data is empty.")
else:
    print("No points were collected.")

In [52]:
vis = meshcat.Visualizer().open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [81]:
import numpy as np
import magnum as mn
import plotly.graph_objs as go

# Function to convert depth and color to point cloud (with transformation)
def depth_to_point_cloud_with_pose(depth, color, fx, fy, cx, cy, translation, rotation):
    height, width = depth.shape
    i, j = np.meshgrid(np.arange(width), np.arange(height), indexing='xy')
    x = (i - cx) * depth / fx
    y = (j - cy) * depth / fy
    z = depth
    valid = depth > 0
    points = np.stack((x[valid], y[valid], z[valid]), axis=-1)
    colors = color[valid] / 255.0  # Normalize color for Plotly

    # Transform points to world coordinates using the agent's pose (translation and rotation)
    transformed_points = []
    R_image2camera = mn.Matrix3x3(np.array([[1, 0., 0], [0, -1, 0], [0, 0, -1]]))
    for point in points:
        point_vec = mn.Vector3(point[0], point[1], point[2])
        # Apply rotation and then translation
        rotated_point = (rotation.to_matrix() @ R_image2camera) * point_vec
        world_point = rotated_point + translation  # Apply translation
        transformed_points.append(world_point)
    
    return np.array(transformed_points), colors


# Initialize variables for point cloud merging
merged_points = []
merged_colors = []
merged_rotation = []
merged_translation = []
count = 0
for observation, translation, rotation in zip(observations, translations, rotations):
    count += 1
    if count % 10 != 0:
        continue
    print(translation, rotation)
    # Convert current depth and color image to point cloud
    points, colors = depth_to_point_cloud_with_pose(
        observation['depth_sensor'], observation['color_sensor_1st_person'][:,:,:3], fx, fy, cx, cy, translation, rotation)
    # Downsample the point cloud
    sample_size = int(len(points) * 0.005)  # Keep 3% of the points
    indices = np.random.choice(len(points), sample_size, replace=False)
    points_sampled = points[indices]
    colors_sampled = colors[indices]
    # Debug: Check if points and colors are generated
    print(f"Frame {count}: {len(points)} points, {len(colors)} colors")

    # Merge point clouds (concatenate the points and colors)
    if len(points) > 0:  # Only add if points are generated
        merged_points.append(points_sampled)
        merged_colors.append(colors_sampled)
        merged_rotation.append(rotation)
        merged_translation.append(translation)


Vector(-2, 0, 0) Quaternion({0, -0.382683, 0}, 0.923879)
Frame 10: 873444 points, 873444 colors
Vector(-2, 0, 0) Quaternion({0, -0.707106, 0}, 0.707106)
Frame 20: 763843 points, 763843 colors
Vector(-2, 0, 0) Quaternion({0, -0.923879, 0}, 0.382683)
Frame 30: 712780 points, 712780 colors
Vector(-1.7, 0, 0) Quaternion({0, -0.999999, 0}, -1.45286e-07)
Frame 40: 796856 points, 796856 colors
Vector(1.3, 0, 0) Quaternion({0, -0.923879, 0}, -0.382683)
Frame 50: 770338 points, 770338 colors
Vector(4, 0, 0.3) Quaternion({0, -0.707106, 0}, -0.707106)
Frame 60: 768086 points, 768086 colors
Vector(4, 0, 3.3) Quaternion({0, -0.382683, 0}, -0.923878)
Frame 70: 681814 points, 681814 colors
Vector(3.9, 0, 5.9) Quaternion({0, -0.0392595, 0}, -0.999227)
Frame 80: 744792 points, 744792 colors
Vector(2.9, 0, 4.9) Quaternion({0, -0.0392595, 0}, -0.999227)
Frame 90: 722980 points, 722980 colors
Vector(1.9, 0, 3.9) Quaternion({0, -0.0392595, 0}, -0.999227)
Frame 100: 715223 points, 715223 colors
Vector(0.900

In [75]:
R_image2camera = mn.Matrix3x3(np.array([[1, 0., 0], [0, -1, 0], [0, 0, -1]]))

point_vec = mn.Vector3(1, 2, 3)
# Apply rotation and then translation
(R_image2camera @ rotation.to_matrix()) * point_vec

Vector(-2.38895, -2, -2.07193)

In [73]:
R_image2camera @ rotation.to_matrix()

Matrix(0.92388, 0, -0.382683,
       0, -1, 0,
       -0.382683, 0, -0.92388)

In [69]:
rotation.to_matrix()

Matrix(0.92388, -0, -0.382683,
       0, 1, -0,
       0.382683, 0, 0.92388)

In [63]:
import meshcat.geometry as g
import meshcat.transformations as tf


def draw_meshcat_frame(path: str, rotation, translation):
    scale = 0.5
    
    # Create the X-axis (red)
    x_axis = np.array([
        [0, scale],  # X-coordinates
        [0, 0],      # Y-coordinates
        [0, 0]       # Z-coordinates
    ])
    vis[f"{path}/x_axis"].set_object(g.LineSegments(g.PointsGeometry(x_axis), g.MeshBasicMaterial(color=0xff0000)))
    
    # Create the Y-axis (green)
    y_axis = np.array([
        [0, 0],      # X-coordinates
        [0, scale],  # Y-coordinates
        [0, 0]       # Z-coordinates
    ])
    vis[f"{path}/y_axis"].set_object(g.LineSegments(g.PointsGeometry(y_axis), g.MeshBasicMaterial(color=0x00ff00)))
    
    # Create the Z-axis (blue)
    z_axis = np.array([
        [0, 0],      # X-coordinates
        [0, 0],      # Y-coordinates
        [0, scale]   # Z-coordinates
    ])
    vis[f"{path}/z_axis"].set_object(g.LineSegments(g.PointsGeometry(z_axis), g.MeshBasicMaterial(color=0x0000ff)))
    
    # Optional: Apply a transformation to the coordinate frame
    rotation_matrix = np.eye(4)
    rotation_matrix[:3, :3] = rotation.to_matrix()
    transform = tf.concatenate_matrices(
        tf.translation_matrix(translation),
        rotation_matrix,
    )
    vis[path].set_transform(transform)

In [82]:
for i, (points, colors, rotation ,translation) in enumerate(zip(merged_points, merged_colors, merged_rotation, merged_translation)):
    vis[f"view{i}/pointcloud"].set_object(g.PointCloud(points.T, colors.T, size=0.03))
    draw_meshcat_frame(f"view{i}/frame", rotation, np.array([translation.x, translation.y, translation.z]))

In [46]:
for i, (rotation ,translation) in enumerate(zip(rotations, translations)):
    draw_meshcat_frame(f"path/view{i}/frame", rotation, np.array([translation.x, translation.y, translation.z]))